
# 第2讲：监督学习与集成学习概论（Python 实操讲义）

本 Notebook 配套“第2讲：正则化建模与集成学习方法”，包含两部分：
1. **LASSO 回归**：系数路径与交叉验证选择超参（详见slides）；
2. **决策树**：分类性能与特征重要性。

运行说明：
- 建议使用 Python 3.9+，依赖 `numpy`、`pandas`、`matplotlib`、`scikit-learn`



## 环境准备（可选）
> 如本地缺失依赖，可执行下列命令（如在 JupyterLab/VS Code 的终端中）：
```bash
pip install numpy pandas matplotlib scikit-learn
```


![](kfold.png)


## 决策树
数据'employee.csv'
共10个变量：
* 因变量left，1表示离职
* 变量satisfaction_level表示员工满意度
* 变量last_evaluation表示最近一次评估得分
* 变量number_project表示参与的项目数量
* 变量average_montly_hours表示工作时间（小时/月）
* 变量time_spend_company表示在公司工作的年限
* 变量Work_accident表示是否发生过工作事故
* 变量promotion_last_5years表示过去5年有没有升迁过
* 变量department表示工作的部门
* 变量salary表示薪资水平：高、中、低

In [17]:
# 数据探索
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tkinter import *
# import pandas_profiling
import dtale
#导入数据
data = pd.read_csv("data/employee.csv")
#进行数据探索
data.info()
print(data.describe())
print(data['department'].value_counts())
print(data['salary'].value_counts())
print(data['left'].value_counts())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   promotion_last_5years  14999 non-null  int64  
 7   department             14999 non-null  object 
 8   salary                 14999 non-null  object 
 9   left                   14999 non-null  int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB
       satisfaction_level  last_evaluation  number_project  \
count        14999.000000     14999.000000    14999.000000   
mean             0.612834         0.716102        3.803054   
std              0.

In [19]:
#训练决策树模型

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tkinter import *

# 导入数据
data_prepared = pd.read_csv("data/employee_prepared.csv")

# 打乱数据
df = data_prepared.sample(frac=1) 
print(df.head()) #查看打乱后的前 5行

# 拆分特征与标签
y = df['left'].copy() # 目标变量（因变量），是否离职（left）
X = df.drop("left", axis=1) # 自变量（特征），除了 left 之外的所有变量
feature_names = X.columns # 保存特征名，方便后续分析或可视化。

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
stratify=y, random_state=42)
# test_size=0.3：30% 作为测试集，70% 作为训练集。
# stratify=y：按 y（是否离职）的分布分层抽样，确保训练集和测试集中的正负样本比例和原始数据一致。
# random_state=42：设置随机种子，保证结果可重复。


       satisfaction_level  last_evaluation  number_project  \
494                  0.84             1.00               5   
10886                0.63             0.71               4   
3459                 0.63             0.62               5   
10531                0.81             0.91               3   
5681                 0.61             0.87               4   

       average_montly_hours  time_spend_company  Work_accident  \
494                     218                   5              0   
10886                   244                   2              0   
3459                    212                   6              0   
10531                   184                   3              0   
5681                    151                   3              0   

       promotion_last_5years  left  salary_new  depart_IT  depart_RandD  \
494                        0     1           2      False         False   
10886                      0     0           1      False         False   
3459 

In [21]:
#训练模型
from sklearn.tree import DecisionTreeClassifier

# 树的最大深度限制为 3
tree_clf = DecisionTreeClassifier(max_depth=3) 
# 在训练集上训练模型
tree_clf.fit(X_train, y_train)

#利用交叉验证评估模型
from sklearn.model_selection import cross_val_score

#cross_val_score：做 K 折交叉验证
tree_scores = cross_val_score(tree_clf, X_test, y_test,
                              scoring="accuracy", cv=5)  

tree_score = tree_scores.mean()
print("决策树模型在验证数据集的准确度: {0:.2f}".format(tree_score))

#看变量的重要性
print("决策树模型变量的重要性：")
for feature, importance in zip(tree_clf.feature_names_in_, tree_clf.feature_importances_):
    print(feature, importance)



决策树模型在验证数据集的准确度: 0.95
决策树模型变量的重要性：
satisfaction_level 0.6024047453941479
last_evaluation 0.1396090581366336
number_project 0.1057045548418406
average_montly_hours 0.004579056261010213
time_spend_company 0.14770258536636774
Work_accident 0.0
promotion_last_5years 0.0
salary_new 0.0
depart_IT 0.0
depart_RandD 0.0
depart_accounting 0.0
depart_hr 0.0
depart_management 0.0
depart_marketing 0.0
depart_product_mng 0.0
depart_sales 0.0
depart_support 0.0
depart_technical 0.0


In [22]:
#输出决策树的图形
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data

dot_data = export_graphviz(tree_clf,
                           out_file=None,
                           feature_names=feature_names)
graph = graph_from_dot_data(dot_data)
graph.write_png('employee_tree.png')


True

![](employee_tree.png)

zsh:1: number expected
